In [1]:
!pip install transformers evaluate accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
import pandas as pd

df =  pd.read_csv("/kaggle/input/reann-model-data/reann_model_data.csv", index_col = False)
df

,review,sentiment
0,Неточность описания и полнейшее несовпадение п...,negative
1,"Заказывала размер xl для Мамы, оказалась жутка...",negative
2,"Плохой сервис, неудобная кровать, невнимательн...",mixed
3,"очень тонкая,просвечивает",negative
4,"Не понравились! Брала чёрные, на фото они плот...",negative
...,...,...
1466,"не стоит она таких денег, нитки везде торчат.",negative
1467,"Товар хороший, но шёл очень долго, я был очень...",mixed
1468,Очень жаркий материал! просто очень! когда м...,mixed
1469,"очень короткая, хоть размер l но длина чуть н...",negative


In [6]:
df["sentiment"].value_counts()

sentiment
negative    879
mixed       557
del          18
neutral      15
Name: count, dtype: int64

In [7]:
df = df.drop(df[df.sentiment == "del"].index)
df = df.drop(df[df.sentiment == "neutral"].index)
df["sentiment"].value_counts()

sentiment
negative    879
mixed       557
Name: count, dtype: int64

In [8]:
df = df.rename(columns={'review': 'text', 'sentiment': 'label'})

In [9]:
import numpy as np

In [21]:
df['text'].replace('', np.nan, inplace=True)
df = df.dropna()

In [22]:
df

,text,label
0,Неточность описания и полнейшее несовпадение п...,negative
1,"Заказывала размер xl для Мамы, оказалась жутка...",negative
2,"Плохой сервис, неудобная кровать, невнимательн...",mixed
3,"очень тонкая,просвечивает",negative
4,"Не понравились! Брала чёрные, на фото они плот...",negative
...,...,...
1466,"не стоит она таких денег, нитки везде торчат.",negative
1467,"Товар хороший, но шёл очень долго, я был очень...",mixed
1468,Очень жаркий материал! просто очень! когда м...,mixed
1469,"очень короткая, хоть размер l но длина чуть н...",negative


In [23]:
from sklearn.model_selection import train_test_split

# разделение на тестовую и тренировочную выборки
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)

In [24]:
import datasets
from datasets import Dataset, DatasetDict

tds = Dataset.from_pandas(train_data)
vds = Dataset.from_pandas(test_data)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds

print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1292
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 144
    })
})


In [14]:
# токенизатор для отзывов
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("blinoff/roberta-base-russian-v0")

In [15]:
# переводим классы из строк в числа
from datasets import ClassLabel

labels = ClassLabel(num_classes = 2,names=["negative", "mixed"])

In [16]:
def tokenize(batch):
    tokens = tokenizer(batch['text'], padding=True, truncation=True)
    tokens['label'] = labels.str2int(batch['label'])
    return tokens

In [17]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [25]:
tokenized_corpus_train = tds.map(tokenize, batched=True)
tokenized_corpus_test = vds.map(tokenize, batched=True)

tokenized_corpus_train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_corpus_test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [54]:
import evaluate

accuracy = evaluate.load("f1")

In [27]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [28]:
# id2label = {0: "negative", 1: "neutral", 2: "positive", 3: "mixed"}
# label2id = {"negative": 0, "neutral": 1, "positive": 2, "mixed": 3}
id2label = {0: "negative", 1: "mixed"}
label2id = {"negative": 0, "mixed": 1}

In [55]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "blinoff/roberta-base-russian-v0",
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    attention_probs_dropout_prob=0.1,
    hidden_dropout_prob=0.1
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at blinoff/roberta-base-russian-v0 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [56]:
# Определение параметров тренировки и тренировка модели
training_args = TrainingArguments(
    report_to=None,
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    evaluation_strategy = "steps",
    save_steps = 200,
    save_total_limit = 5,
    logging_steps=60,
    learning_rate=2e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_corpus_train,
    eval_dataset=tokenized_corpus_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,F1
60,0.569800,0.504535,0.534884
120,0.566600,0.500638,0.688525
180,0.495600,0.547464,0.689655
240,0.412200,0.445392,0.744681
300,0.429100,0.392228,0.761905
360,0.366500,0.358674,0.816327
420,0.303100,0.367240,0.759259
480,0.259200,0.344861,0.776699


TrainOutput(global_step=486, training_loss=0.42252238248110796, metrics={'train_runtime': 239.6813, 'train_samples_per_second': 16.171, 'train_steps_per_second': 2.028, 'total_flos': 1019818450575360.0, 'train_loss': 0.42252238248110796, 'epoch': 3.0})

In [57]:
trainer.save_model("/kaggle/working/reann_model")

# F1-score evaluation on test dataset

In [37]:
from transformers import pipeline
from tqdm.notebook import tqdm

sa = pipeline(task='sentiment-analysis', model = model, tokenizer = tokenizer, max_length=512, truncation=True, device = 0)
pred = []

In [62]:
test_data_list = test_data["text"].to_list()

In [ ]:
for sentence in tqdm(test_data_list):
    out = sa(sentence)
    pred.append(out)

In [ ]:
# сохраняю тексты и предсказания в файл
dic = {'text':test_data_list, 'label':pred}
output_df = pd.DataFrame(dic)
output_df.to_csv('sec_model_output.tsv', sep = '\t')

In [ ]:
# оставляю от предсказаний только label (до этого было то, что возвращала модель, типа "[{'label': 'neutral', 'score': 0.7671204209327698}]")
pred_for_f1 = []
for i in output_df['label']:
  #l = i[0]
  pred_for_f1.append(i[0]['label'])
pred_for_f1[:10]

In [ ]:
target = list(test_data['label'])

In [76]:
from sklearn.metrics import f1_score
f1 = f1_score(target, pred_for_f1, average=None)
f1

array([0.79166667, 0.89583333])

# Reannotation

In [80]:
reann_df = pd.read_csv("/kaggle/input/reann-neg/4secondmodel_1.tsv", sep="\t", index_col = False)

In [81]:
reann_df['text'].replace('', np.nan, inplace=True)
reann_df = reann_df.dropna().drop(["Unnamed: 0", "target"], axis = 1)

In [82]:
reann_df

,text
0,Качество ткани ГОВНО!!! 100% синтетика
1,Пришло без точного адреса.С почты звонили на с...
2,Материал ужас просто
3,ткань не такая как смотрится на фото- просвечи...
4,"очень неприятная к телу ткань,страшно электрил..."
...,...
5689,ткань не соответствует фото
5690,"Отель мне не понравился, единственный плюс ряд..."
5691,белый топ действительно очень хорошо просвечив...
5692,"Рубашка оказалась мала, хотя размеры проводила..."


In [91]:
from transformers import pipeline
from tqdm.notebook import tqdm

sa = pipeline(task='sentiment-analysis', model = model, tokenizer = tokenizer, max_length=512, truncation=True, device = 0)
pred = []

In [93]:
reann_list = reann_df["text"].to_list()

In [95]:
for sentence in tqdm(reann_list):
    out = sa(sentence)
    pred.append(out)

  0%|          | 0/5694 [00:00<?, ?it/s]

In [102]:
len(dic_reann["label"])

5694

In [103]:
len(dic_reann["text"])

5694

In [105]:
# сохраняю тексты и предсказания в файл
dic_reann = {'text':reann_list, 'label':pred}
output_df = pd.DataFrame(dic_reann)
pred_for_f1 = []
for i in output_df['label']:
    pred_for_f1.append(i[0]['label'])
output_df['label'] = pred_for_f1
output_df.to_csv('reannotated.tsv', sep = '\t')